# Instalação

In [ ]:
#!pip install xcompact3d_toolbox

# Parâmetros

Os parâmetros físicos e computacionais são herdados por [xcompact3d_toolbox.Parameters](https://docs.fschuch.com/xcompact3d_toolbox/references/api-reference.html#xcompact3d_toolbox.parameters.Parameters). É programado sobre [Traitlets](https://traitlets.readthedocs.io/en/stable/index.html) o que permite que os parâmetros sejam compatíveis com XCompact3d which aims to make the parameters compatible with what XCompact3d. Além disso, a biblioteca oferece:

* Verificação por tipo;
* Valores padrão, restrições e dependências entre parâmetros são aplicados quando necessários;
* Controles interativos com [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/).

In [ ]:
import numpy as np

import xcompact3d_toolbox as x3d

Primeiro defini-se a precisão numérica. Verifique no Makefile qual a precisão que o Xcompact3d foi copilado, use `np.float64` se `-DDOUBLE_PREC` foi utilizado, `np.float32` para os demais.

In [ ]:
x3d.param["mytype"] = np.float32

## Inicialização

Há algumas formas de inicializar a classe. Pode-se instancia-lá com os valores padrões ao não se passar nenhum argumento.

In [ ]:
prm = x3d.Parameters()

Na [documentação da Api](https://docs.fschuch.com/xcompact3d_toolbox/references/api-reference.html) pode-se ver uma lista com todas as variáveis.

Também pode-se visualizar na tela com o comando `print`:

In [ ]:
print(prm)

Também pode-se visualizar valores individualmente:

In [ ]:
# Número de Reynolds
print(prm.re)

# Atribuir um valor novo
prm.re = 1e6
print(prm.re)

Pode-se inicializar alguns parâmetros e utilizar o pdrão para o restante:

In [ ]:
prm = x3d.Parameters(
    filename="example.i3d",
    itype=10,
    nx=129,
    ny=65,
    nz=32,
    xlx=15.0,
    yly=10.0,
    zlz=3.0,
    nclx1=2,
    nclxn=2,
    ncly1=1,
    nclyn=1,
    nclz1=0,
    nclzn=0,
    iin=1,
    istret=2,
    re=300.0,
    init_noise=0.0125,
    inflow_noise=0.0125,
    dt=0.0025,
    ifirst=1,
    ilast=45000,
    irestart=0,
    icheckpoint=45000,
    ioutput=200,
    iprocessing=50,
)

Pode-se salvar a estrutura de dados em arquivo texto com o comando:

In [ ]:
prm.write()

Por último pode-se carregar os parâmetros de um arquivo texto:

In [ ]:
prm = x3d.Parameters(filename="example.i3d")
prm.load()

A mesma operação pode-se ser feita de uma forma mais concisa:

In [ ]:
prm = x3d.Parameters(loadfile="example.i3d")

Também pode-se carregar um arquivo de um formato antigo ([mais informações estão disponíveis nesse link](https://github.com/fschuch/xcompact3d_toolbox/issues/7)):

``` python
prm = x3d.Parameters(loadfile="incompact3d.prm")
```

Pode-se instanciar a malha pelo método `get_mesh`.

In [ ]:
prm.get_mesh()

## Traitlets

### Verificação de Tipo

Todos os parâmetros são verificados, para garantir que eles são compatíveis com o que `XCompact3d` espera. Rode na próxima célula os casos a baixo e observe o erro retornado.

```python
prm.itype = 10.5
prm.itype = -5
prm.itype = 20
prm.itype = 'sandbox'

```


In [ ]:
prm.itype = 'sandbox'

### Validação

Some parameters, like mesh points (`nx`, `ny` and `nz`), trigger a validation operation when a new value is attributed to them.
Due to restrictions at the FFT library, they must be equal to:

Alguns parâmetros, como os pontos da malha (`nx`, `ny` e `nz`), aciona uma operação de validação quando um novo valor é atribuido a eles. Devido as restrições da librária FFT, valores válidos seguem a seguinte restrição:

$$
n_i = \left\{ \begin{array}{ll} 2^{1+a} \times 3^b \times 5^c &\mbox{se periódico,} \\ 2^{1+a} \times 3^b \times 5^c + 1 &\mbox{para os outros casos,}
\end{array} \right.
$$

onde  $a$, $b$ e $c$ são inteiros não negativos. Além disso, o esquema de derivação impõem:

$$
n_i \ge \left\{ \begin{array}{ll} 8 &\mbox{se periódico,} \\ 9 &\mbox{para os outros casos.}
\end{array} \right.
$$

Novamente tente os seguintes valores na célula a baixo:

```python
prm.nx = 129
prm.nx = 4
prm.nx = 60
prm.nx = 61
```

In [ ]:
prm.nx = 61

### Observação

Outros parâmetros, como a resolução da malha (`dx`, `dy` and `dz`), são atualizados automaticamente quando novos valores são atribuidos ao domínio. Isso é verificado pela função abaixo:

In [ ]:
def show_param():
    for var in "nclx1 nclxn nx xlx dx".split():
        print(f"{var:>5} = {getattr(prm, var)}")

Observe o que a função produz na tela:

In [ ]:
show_param()

Note o que acontece quando se muda o comprimento em x:

In [ ]:
prm.xlx = 50.0

show_param()

`dx` foi atualizado automaticamente. Veja o que ocorre quando se altera o  número de pontos:

In [ ]:
prm.nx = 121

show_param()

Novamente a resolução foi modificada. Agora `xlx` será atualizada para satisfazer uma nova resolução:

In [ ]:
prm.dx = 1e-2

show_param()

Condições de contorna são presentes também. Xcompact3d oferece três opções para o campo de velocidade:

* Periódico `0`;
* Deslizamento-livre `1`;
* Dirichlet `2`.

Eles podem ser atribuidos individualmente a cada superfície:

* `nclx1` e `nclxn`, onde $x=0$ e $x=xlx$;
* `ncly1` e `nclyn`, onde $y=0$ e $y=yly$;
* `nclz1` e `nclzn`, onde $z=0$ e $z=zlz$.

O que leva a 5 possibilidades (`00`, `11`, `12`, `21` and `22`), devido que for aplicado periodicidade, ela deve ser imposta em ambas as paredes.

Testando-se novamente com:

In [ ]:
show_param()

Modificando uma das interfaces para ser periódico (`nclx1 = 0`), por consistência, o outro lado deve ser também.

In [ ]:
prm.nclx1 = 0

show_param()

Agora, impondo deslizamento-livre em um dos lados (`nclx1 = 1`), o outro deve não ser periódico:

In [ ]:
prm.nclx1 = 1

show_param()

Definindo a outra interface com periódico:

In [ ]:
prm.nclxn = 0

show_param()

Retornando para Dirichlet:

In [ ]:
prm.nclxn = 2

show_param()

Deslizamento-livre, novamente:

In [ ]:
prm.nclxn = 1

show_param()

`nclx1` não precisou ser atualizado, já que as condições `1` e `2` podem ser combinadas livremente. Perceba como `nx` foi alternado entre 121 e 120 dependendo da condição de contorno imposta. `dx` e `xlx` permanecem inalterados.

## Interface Gráfica

A classe `ParametersGui` é uma subclasse de `Parameters`, incluindo todas as funcionalidades descritas anteriormente. `ParametersGui` oferece uma interface de usuário com [IPywidgets](https://ipywidgets.readthedocs.io/en/stable/index.html).

Esse parte ainda está em desenvolvimento, novas funcionalidades serão implementadas em breve.

Comece com:

In [ ]:
prm = x3d.ParametersGui()

Widgets são retornadas sobdemanda para qualquer instância da `classe ParametersGui` quando é chamada:

In [ ]:
prm("nx", "xlx", "dx", "nclx1", "nclxn")

Fique testando diferentes valores no widgets e veja como os parâmetros são atualizados automaticamente.

Chamando a função `show_param` pode-se verificar como os valores no widgets são salvos na estrutura de dados.

In [ ]:
show_param()

Refaça esse processo e chame a função `show_param` novamente.

Se alterarmos o valor de um parâmetro como na célula abaixo os widgets são atualizados altomaticamente.

In [ ]:
prm.nclx1 = 0

Diferentes widgets são sempre sincronizados, observe como modificando o widgets a baixo o parâmetro é atualizado no widgets anterior.

In [ ]:
prm("nx")

O último exemplo é sobre decomposição de domínio para computação paralela, Xcompact3d usa [2DECOMP&FFT](http://www.2decomp.org/).
Os parâmetros `p_row` e `p_col` são apresentados como função do número de núcleos computacionais `ncores`, com `p_row * p_col = ncores`, com `p_row * p_col = 0` sendo o modo auto-tuning.

In [ ]:
prm("ncores", "p_row", "p_col")

Por fim, pode-se ver o que acontece quando a `classe ParametersGui` é plotada na tela, a descrição de cada atributo pode ser vista posicionando o mouse sobre a variável.

In [ ]:
prm